<h2 align='center'> ML Course: ML Flow Tutorial</h2>

In [36]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [37]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [38]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [39]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [40]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [41]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [42]:
from imblearn.combine import SMOTETomek

In [43]:
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train,y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [44]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [45]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [46]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [47]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9607142857142857,
   'recall': 0.9962962962962963,
   'f1-score': 0.9781818181818182,
   'support': 270.0},
  '1': {'precision': 0.95,
   'recall': 0.6333333333333333,
   'f1-score': 0.76,
   'support': 30.0},
  'accuracy': 0.96,
  'macro avg': {'precision': 0.9553571428571428,
   'recall': 0.8148148148148149,
   'f1-score': 0.8690909090909091,
   'support': 300.0},
  'weighted avg': {'precision': 0.9596428571428572,
   'recall':

In [48]:
import mlflow

In [49]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("Anomaly_Detection")

for i, elements in enumerate(models):
    model_name = elements[0]
    model_param = elements[1]
    model = elements[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(model_param)
        mlflow.log_metrics({
            "accuracy": report['accuracy'],
            "recall_class_1": report['1']['recall'],
            "recall_class_2": report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model,'model')

        else:
            mlflow.sklearn.log_model(model,'model')

2025/11/11 13:04:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 13:04:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 13:04:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/874460147248616545/runs/720f3d2a362e4af4a9e19cba0ab3b45d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/874460147248616545


2025/11/11 13:04:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 13:04:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/874460147248616545/runs/801519d8dd51423e9564149f32183d97
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/874460147248616545


2025/11/11 13:04:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 13:05:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/874460147248616545/runs/9fadeb5710ac41b3a94188cf784b6032
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/874460147248616545


2025/11/11 13:05:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/874460147248616545/runs/745c881fea0146f1ae9e096bc1bf619e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/874460147248616545


### Register the Model

In [50]:
model_name = "XGB-SMOTE"
run_id = input("Please type RunID")
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri,name=model_name)

Please type RunID 745c881fea0146f1ae9e096bc1bf619e


Registered model 'XGB-SMOTE' already exists. Creating a new version of this model...
2025/11/11 13:09:16 WARNING mlflow.tracking._model_registry.fluent: Run with id 745c881fea0146f1ae9e096bc1bf619e has no artifacts at artifact path 'model', registering model based on models:/m-39a3cc9b2cf947f0a4b64b64f5887e7e instead
2025/11/11 13:09:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-SMOTE, version 1


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/874460147248616545/runs/745c881fea0146f1ae9e096bc1bf619e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/874460147248616545


Created version '1' of model 'XGB-SMOTE'.


### Load the Model

In [56]:
model_version = 1
model_uri=  f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

In [59]:
model_version = 1
model_uri=  f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

### Transition the Model to Production

In [61]:
current_model_uri = f"models:/{model_name}@challenger"
productiion_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri,dst_name=productiion_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1762849016187, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762849016187, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='745c881fea0146f1ae9e096bc1bf619e', run_link='', source='models:/XGB-SMOTE/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [62]:
model_uri=  f"models:/{productiion_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 